# SingleCompartment

## 电生理特性

`SingleCompartment` 表示一个没有空间结构的点神经元模型，用于对具有单个房室的的基于电导的神经元进行建模。膜电位的变化由以下微分方程控制：

$$
C_m \frac{dV}{dt} = \sum_j g_j(E_j - V) + I_{\text{ext}}
$$

其中：

- $C_m$：膜电容
- $V$：膜电位
- $g_j$：第 $j$ 个离子通道的电导
- $E_j$：该通道的反转电位
- $I_{\text{ext}}$：外部注入电流

每个通道的电导 $g_j$ 由激活门控变量和失活门控变量控制，形式为：

$$
g_j = \bar{g}_j \cdot M^x \cdot N^y
$$

其中：

- $\bar{g}_j$：最大电导
- $M$：激活变量
- $N$：失活变量
- $x, y$：幂指数，具体由通道类型决定

这些门控变量的动态遵循一阶动力学方程：

$$
\frac{dx}{dt} = \phi_x \cdot \frac{x_\infty(V) - x}{\tau_x(V)}
$$

也可以等价地写作：

$$
\frac{dx}{dt} = \phi_x \left( \alpha_x (1 - x) - \beta_x x \right)
$$

其中：

- $x \in \{M, N\}$
- $\phi_x$：温度因子
- $x_\infty$：稳态值
- $\tau_x$：时间常数
- $\alpha_x$ 和 $\beta_x$：速率常数

## 建模实现

`SingleCompartment` 的建模实现相当简单，你只需要继承 `HHTypedNeuron` 并对其中不同离子通道进行选型设定即可。

下面我们来看一个例子:

```python
class HH(braincell.SingleCompartment):
    def __init__(self, size, solver='rk4'):
        super().__init__(size, solver=solver)

        self.na = braincell.ion.SodiumFixed(size, E=50. * u.mV)
        self.na.add(INa=braincell.channel.INa_HH1952(size))

        self.k = braincell.ion.PotassiumFixed(size, E=-77. * u.mV)
        self.k.add(IK=braincell.channel.IK_HH1952(size))

        self.IL = braincell.channel.IL(
            size,
            E=-54.387 * u.mV,
            g_max=0.03 * (u.mS / u.cm **2)
        )
```

在例程中，我们使用 `SingleCompartment` 构建基于 HH 模型的单神经元，该神经元包含钠通道 `INa`、钾通道 `IK` 和漏电流 `IL` ，这些通道共同决定神经元的放电特性。
当然，这些相应的通道在后续的章节我们也会进行详细的讲解。

通过例程，不难看出， `SingleCompartment` 作为 `braincell` 中常用的细胞建模接口，极大地简化了神经元的构建，大大提升了建模效率。